Shell model
===

Geometric model and meshing. left surface only needed for boundary conditions.

In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.internal import visoptions
import netgen.gui
%gui tk

order = 2

geo = CSGeometry()
cyl = Cylinder (Pnt(0,0,0),Pnt(1,0,0),0.4).bc("cyl")
left = Plane (Pnt(0,0,0), Vec(-1,0,0)).bc("left")
finitecyl = cyl * left * Plane(Pnt(1,0,0), Vec(1,0,0))
geo.AddSurface (cyl, finitecyl)
geo.AddSurface (left, finitecyl)

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
mesh.Curve(order+1)
Draw (mesh)

Fields are: displacement $u \in [H^1(S)]^3$, rotation $\beta \in H(\text{curl},S)$, momentum $\sigma \in H(\text{div div},S)$

In [ ]:
fes1 = HDivDivSurface(mesh, order=order)
fes2 = H1(mesh, order=order+1, dirichlet="left")
fes3 = HCurl(mesh, order=order, dirichlet="left")
fes = FESpace( [fes1,fes2,fes2,fes2,fes3] )

sigma,u1,u2,u3,beta = fes.TrialFunction()
sigma, beta = sigma.Trace(), beta.Trace()

gradu = CoefficientFunction((grad(u1),grad(u2),grad(u3)),dims=(3,3))

nsurf = specialcf.normal(3)
t = specialcf.tangential(3)
nel = Cross(nsurf, t)

ngradu = gradu.trans*nsurf
sigman = sigma*nel

def tang(u):
    return (u*t)*t

thickness = 0.1

Membrane energy with primal method for displacement vector $u$:
$$
\|E_{tt}(u)\|^2_{L^2(S)} 
$$
Shear energy, $\beta$ a rotation field in tangential plane:
$$
\int_S | n^T \nabla u - \beta |^2 
$$
Bending energy (with TDNNS mixed method):
$$
-\frac{1}{2t^2}a(\sigma,\sigma)+b(\beta,\sigma)
$$

In [ ]:
a = BilinearForm(fes, symmetric=True)
#bending energy
a += SymbolicEnergy( -0.5/thickness**2*InnerProduct(sigma,sigma), BND )
a += SymbolicEnergy( -div(sigma)*beta, BND )
a += SymbolicEnergy( sigman * tang(beta), BND, element_boundary=True )
#shearing energy
a += SymbolicEnergy( (ngradu-beta)*(ngradu-beta), BND )

Ptau = Id(3) - OuterProduct(nsurf,nsurf)
Ftau = gradu + Ptau
Ctautau = Ftau.trans * Ftau
Etautau = 0.5*(Ctautau - Ptau)
#membran energy
a += SymbolicEnergy( InnerProduct(Etautau, Etautau), BND )

factor = Parameter(0.1)
a += SymbolicEnergy( -factor*y * u2, BND )

gfu = GridFunction(fes)

uvec = CoefficientFunction(gfu.components[1:4])

Draw(gfu.components[4], mesh, "rotations")
Draw(uvec, mesh, "disp")

solsigma = CoefficientFunction([gfu.components[0]], None)
Draw(solsigma, mesh, "sigma", sd=4)
visoptions.scalfunction = "rotations:0"
visoptions.vecfunction='disp'
SetVisualization(deformation=True)

Increase the load step-wise, solve the non-linear problem by Newton's method. First and second order derivatives are computed by automatic differentiation.

In [ ]:
from newtonsolver import NewtonMinimization

for loadstep in range(8):
    print ("loadstep ", loadstep)
    factor.Set (1.5*(loadstep+1))
    with TaskManager():
        NewtonMinimization (a, gfu)
    Redraw(blocking=True)